In [ ]:
!pip install autorec

# Neural Matrix Factorization (NeuMF)
*Click-through rate*

https://arxiv.org/pdf/1708.05031.pdf

Description:
NeuMF captures the personalized ranking task with implicit feedback with the use of both linearity and non-linearity. NeuMF consists of two subnetwork GMF and DNN layers. GMF layer computes the elementwise product of user and item latent factors. DNN layers concatenates user and item factors then uses multi-layer perceptron (MLP). Combining linearity and non-linearity of models, NeuMF leverages advantages of both subnetwork: generality and flexibility.

## A simple example below

**Step 0:Imports and Configurations**

First, handle the imports with the correct configurations set. Also include the logging settings here.

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "6"

import logging
import tensorflow as tf
from autorecsys.auto_search import Search
from autorecsys.pipeline import Input, LatentFactorMapper, MLPInteraction, PointWiseOptimizer
from autorecsys.pipeline.interactor import InnerProductInteraction
from autorecsys.recommender import CTRRecommender
from autorecsys.pipeline.preprocessor import CriteoPreprocessor


# logging setting
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)



**Step 1: Preprocess data**

CriteoPreprocessor() is a already built-in preprocessor for the Criteo dataset. 


In [ ]:
# Step 1: Preprocess data
criteo = CriteoPreprocessor()  # the default arguments are setup to preprocess the Criteo example dataset
train_X, train_y, val_X, val_y, test_X, test_y = criteo.preprocess()
train_X_numerical, train_X_categorical = criteo.get_x_numerical(train_X), criteo.get_x_categorical(train_X)
val_X_numerical, val_X_categorical = criteo.get_x_numerical(val_X), criteo.get_x_categorical(val_X)
test_X_numerical, test_X_categorical = criteo.get_x_numerical(test_X), criteo.get_x_categorical(test_X)
numerical_count = criteo.get_numerical_count()
categorical_count = criteo.get_categorical_count()
hash_size = criteo.get_hash_size()


**Step 2.0: Build the recommender**

Creates the pipeline

Models can be customized for a searchable recommender

**Step 2.1: Setup mappers to handle inputs**

This step is to build a input node for your data. This examples requires to embed into two parts: gmf & mlp. 


In [ ]:
input = Input(shape=[criteo.get_categorical_count()])
user_emb_gmf = LatentFactorMapper(feat_column_id=0,
                                  id_num=10000,
                                  embedding_dim=64)(input)
item_emb_gmf = LatentFactorMapper(feat_column_id=1,
                                  id_num=10000,
                                  embedding_dim=64)(input)

user_emb_mlp = LatentFactorMapper(feat_column_id=0,
                                  id_num=10000,
                                  embedding_dim=64)(input)
item_emb_mlp = LatentFactorMapper(feat_column_id=1,
                                  id_num=10000,
                                  embedding_dim=64)(input)



**Step 2.2: Setup interactors to handle models**

This example has two interactors. Any of other interactors can be stacked of your choice. 



In [ ]:
innerproduct_output = InnerProductInteraction()([user_emb_gmf, item_emb_gmf])
mlp_output = MLPInteraction()([user_emb_mlp, item_emb_mlp])



**Step 2.3: Setup optimizer to handle the target task**

Two interactors (innerproduct and mlp) are concatenated and optimized.

In [ ]:
output = PointWiseOptimizer()([innerproduct_output, mlp_output])
model = CTRRecommender(inputs=input, outputs=output)


**Step 3: Build the searcher**

This provides the search algorithm


In [ ]:
searcher = Search(model=model,
                  tuner='random',  # hyperband, greedy, bayesian
                  tuner_params={"max_trials": 5}
                  )


**Step 4: Use the searcher to search the recommender**

Search the best model and validate accuracy of the model


Inputs such as objective, batch_size, call backs can be customized for you own model.


In [ ]:
searcher.search(x=[criteo.get_x_categorical(train_X)],
                y=train_y,
                x_val=[criteo.get_x_categorical(val_X)],
                y_val=val_y,
                objective='val_BinaryCrossentropy',
                batch_size=256,
                epochs = 2,
                callbacks = [ tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=1)] 
                )

**Step 5: Evaluate the searched model**


In [ ]:
logger.info('Predicted Ratings: {}'.format(searcher.predict(x=[criteo.get_x_categorical(val_X)])))
logger.info('Predicting Accuracy (mse): {}'.format(searcher.evaluate(x=[criteo.get_x_categorical(val_X)], y_true=val_y)))
